In [8]:
import sympy as sp

In [9]:
# State variables
x_SoC, T_c, T_h = sp.symbols('x_SoC T_c T_h')

# Input variables
I_HP, I_FAN = sp.symbols('I_HP I_LED')

# Battery parameters
Q_max, R_in, P_rest = sp.symbols('Q_max R_in P_rest')
a3, a2, a1, a0 = sp.symbols('a3 a2 a1 a0')
U_oc = a3 * x_SoC**3 + a2 * x_SoC**2 + a1 * x_SoC + a0

# Fan parameters
U_FAN, x_FAN = sp.symbols('U_FAN x_FAN')

# LED parameters
U_LED, I_LED, x_LED, T_LED = sp.symbols('U_LED I_LED x_LED T_LED')

# Peltier module parameters
S_M, R_M, K_M = sp.symbols('S_M R_M K_M')

# Thermal
m_h, cp, m_c, R_eq, R3_lambda, T_inf = sp.symbols('m_h c_p m_c R_eq R3_lambda T_inf')

# Define the three nonlinear differential equations
dTh_dt = (1 / (m_h * cp)) * (S_M * I_HP * T_c - (R_in + 0.5 * R_M) * I_HP**2 - K_M * T_h + K_M * T_c + U_oc * I_HP - R_in * (I_FAN + I_LED) * I_HP + T_inf / R_eq - T_h / R_eq)
dTc_dt = (1 / (m_c * cp)) * (T_LED / R3_lambda - (1 / R3_lambda + K_M) * T_c - S_M * I_HP * T_c + 0.5 * R_M * I_HP**2 + K_M * T_h)
dxSoC_dt = (-1 / (2 * Q_max * R_in)) * (U_oc - sp.sqrt(U_oc**2 - 4 * R_in * (P_rest + I_FAN * U_FAN * x_FAN - R_in * I_HP**2 + I_LED * x_LED * U_oc + I_HP * U_oc - R_in * (I_FAN + I_LED) * I_LED * x_LED - R_in * (I_FAN + I_LED) * I_HP)))

# Define state and input vectors
x = sp.Matrix([x_SoC, T_c, T_h])
u = sp.Matrix([I_HP, I_FAN])

# Define the system of differential equations as a matrix
f = sp.Matrix([dxSoC_dt, dTc_dt, dTh_dt])

# Compute Jacobians for A and B matrices
A = f.jacobian(x)
B = f.jacobian(u)

In [10]:
# Simplified version
# Define state and input vectors
x = sp.Matrix([T_c, T_h])
u = sp.Matrix([I_HP])

# Define the system of differential equations as a matrix
f = sp.Matrix([dTc_dt, dTh_dt])

# Compute Jacobians for A and B matrices
A = f.jacobian(x)
B = f.jacobian(u)

In [11]:
A_latex = sp.latex(A)
B_latex = sp.latex(B)

# Print LaTeX code
print("LaTeX representation of Matrix A:")
print(A_latex)

print("\nLaTeX representation of Matrix B:")
print(B_latex)


LaTeX representation of Matrix A:
\left[\begin{matrix}\frac{- I_{HP} S_{M} - K_{M} - \frac{1}{R_{3 \lambda}}}{c_{p} m_{c}} & \frac{K_{M}}{c_{p} m_{c}}\\\frac{I_{HP} S_{M} + K_{M}}{c_{p} m_{h}} & \frac{- K_{M} - \frac{1}{R_{eq}}}{c_{p} m_{h}}\end{matrix}\right]

LaTeX representation of Matrix B:
\left[\begin{matrix}\frac{1.0 I_{HP} R_{M} - S_{M} T_{c}}{c_{p} m_{c}}\\\frac{- 2 I_{HP} \left(0.5 R_{M} + R_{in}\right) - 2 I_{LED} R_{in} + S_{M} T_{c} + a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}}{c_{p} m_{h}}\end{matrix}\right]


In [12]:
A

Matrix([
[(-I_HP*S_M - K_M - 1/R3_lambda)/(c_p*m_c),             K_M/(c_p*m_c)],
[               (I_HP*S_M + K_M)/(c_p*m_h), (-K_M - 1/R_eq)/(c_p*m_h)]])

In [13]:
B

Matrix([
[                                                                       (1.0*I_HP*R_M - S_M*T_c)/(c_p*m_c)],
[(-2*I_HP*(0.5*R_M + R_in) - 2*I_LED*R_in + S_M*T_c + a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3)/(c_p*m_h)]])

In [18]:
# x_SoC_ss, T_c_ss, T_h_ss = sp.symbols('x_SoC_ss T_c_ss T_h_ss')
# I_HP_ss, I_FAN_ss = sp.symbols('I_HP_ss I_LED_ss')
T_c_ss, T_h_ss = sp.symbols('T_c_ss T_h_ss')
I_HP_ss = sp.symbols('I_HP_ss')

# x_ss = sp.Matrix([x_SoC_ss, T_c_ss, T_h_ss])
x_ss = sp.Matrix([T_c_ss, T_h_ss])
# u_ss = sp.Matrix([I_HP_ss, I_FAN_ss])
u_ss = sp.Matrix([I_HP_ss])

steady_state_linear_eq = A * sp.Matrix([x_ss]) #+ B * sp.Matrix([u_ss])

# Solve linear steady-state
linear_steady_state = sp.solve(steady_state_linear_eq, [x_ss], dict=True)
print("\nLinear steady-state solutions:")
sp.pprint(linear_steady_state)


Linear steady-state solutions:
[]


In [15]:
# # Set derivatives to zero for steady-state
# steady_state_eqs = [eq for eq in f]

# # Solve steady-state equations
# steady_state = sp.solve(steady_state_eqs, [x_SoC, T_c, T_h], dict=True)
# print("Nonlinear steady-state solutions:")
# sp.pprint(steady_state)